Back in Part 2, we tried building our Q updating function in two ways: as an iterative algorithm, and as an agent method. The iterative algorithm seemed to work ok, but it added Q indexes that wouldn't happen because they were after the terminal state. I'm not sure if that's a problem, really, except that if we're testing for convergence with something like MSR, it might throw the result off. Maybe not.

Since then I've taken a look at some other game space implementations (needs reference to OpenAIGym blackjack.py environemtn), and Q-Learning algorithms (needs reference to https://gist.github.com/awjuliani/9024166ca08c489a60994e529484f7fe), and I think that the iterative approach wasn't bad after all. Let's go back to it.

To review, this is what we hope to implement:

```
Initialize s
Repeat
    Choose a
    Take action a, observe r, s'
    Update Q(s,a)
    s <-- s'
Until s is terminal
```
Let's get to work. To do:

* [x] We need to create the training loop for Q(s,a)

   The training loop until now has been selecting actions which are purely random. This is an exploration-only policy, and it's a valid approach, but it doesn't help us converge on an optimal policy because there's no benefit of experience, or exploitation. 


* [x] So we need an exploration function

   This is a function that decides when to choose randomly and when to choose based on past knowledge.
   * We can simply try something random every n ticks of epoch
   * We can introduce a threshold of exploration, which over time will decrease. As the probability that we've learned all we need to learn about a certain situation increases, because of the number of times we've visited it, we will go with what we know instead of trying something different. The only question now is, do we calculate the threshold based on state, or based on state,action?


* [ ] We need a performance-measuring function. There are two choices:
   * A loss function using expected $Q(s,a)=R(s) + \gamma \underset{a'}{\operatorname{max}}Q(s',a')$
   * A discounted total reward-to-go function $ R(t) = \sum_{t=0}^{N} \gamma^t r_t $, where t is any given time step in a run

In [61]:
# %load main.py
from player import Player
from shoe import Shoe
from utilities import hit, newHand, deal, getReward, getAction, getUpdatedQsa
from collections import defaultdict
from IPython.display import clear_output

#import logging
#logging.basicConfig(filename='LearningPolicy.log', filemode='w', level=logging.DEBUG, format='%(asctime)s %(levelname)s:%(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')

# Initialize
shoe = Shoe(1)
dealer = Player()
player = Player()
Q = defaultdict(float)
N = defaultdict(float)
RTG = [] # Reward to Go function
CR = [] # Cumulative reward function
LOSS = [] # Loss function
ACTIONS=('HIT','STAND',)
epsilon = 10
lr = 0.08
discount = 0.99

Overwriting main.py


In [38]:
%matplotlib inline

In [60]:
# %load round.py
import matplotlib.pyplot as plt
import numpy as np
import random
import logging

logging.basicConfig(filename='LearningPolicy.log', filemode='w', level=logging.INFO, format='%(asctime)s %(levelname)s:%(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')

# Updated getAction() using exploration/exploitation
def getAction(Q,N,s):
    e = epsilon/(epsilon + N[s])
    rr = random.random()
    if rr < e:
        logging.debug("Exploring (N[s] = {}, e = {}, rr = {}): Random action selected".format(N[s],e,rr))
        return ACTIONS[random.randint(0,1)]
    else:
        # Use what we've learned
        logging.debug("Exploiting (N[s] = {}, e = {}, rr = {}): Optimal action selected".format(N[s],e,rr))
        if Q[s+('HIT',)] > Q[s+('STAND',)]:
            return 'HIT'
        else:
            return 'STAND'

def getUpdatedQsa(Q,sa,r,s1,A):
    return Q[sa] + 0.08*(r + discount*max(Q[s1+A[0:1]],Q[s1+A[1:2]]) - Q[sa])

def discountedRewardToGo(R):
    pass

# Intitalize first!
# Train with a number of hands
bankroll = 500.00
netWins = 0
wins = 0
losses = 0
pushes = 0
WALLET=[]
RMS=[]
R=[0]
n = 10000

# print("Starting bankroll: {}".format(bankroll))
for i in range(n):
    # Initialize s
    newHand(dealer,player,shoe)
    s = (player.getPoints(),dealer.hand[0],)
    done = False
    while True:
        logging.debug("Current state: {}".format(s))
        # choose an action
        a = getAction(Q,N,s)
        logging.debug("Player {}: ".format(a))

        # Take the action
        if a == 'HIT':
            hit(player,shoe)
            if player.getPoints() > 21:
                done = True
        else:
            # Player stands; play out the dealer
            while dealer.getPoints() < 17:
                logging.debug("Dealer HIT: ")
                hit(dealer,shoe)
            done = True
        s1 = (player.getPoints(),dealer.hand[0],)

        # Calculate the reward
        if not done:
            r = 0
        else:
            r = getReward(player,dealer)

        # Update Q(s,a)
        Q[s+(a,)] = getUpdatedQsa(Q,s+(a,),r,s1,ACTIONS)
        N[s] += 1
        s = s1    

        # Stop if we're at the terminal state
        if done:
            break

    # Calculate and keep track of wins/losses
    if r < 0:
        logging.debug("Lose ({})".format(r))
        losses += 1
    elif r > 0:
        logging.debug("Win ({})".format(r))
        wins += 1
    else:
        logging.debug("Push ({})".format(r))
        pushes += 1

    # Data from the episode    
    bankroll += r*5
    WALLET.append(bankroll)
    R.append(r)
    RMS.append(np.sqrt(sum((np.fromiter(iter(Q.values()), dtype=float))**2)/len(Q)))
    RTG.append(sum([(R[t]*(discount**t)) for t in range(len(R))]))
    CR.append(sum(R))

logging.info("\n\tWins: {}\n\tLosses: {}\n\tPushes: {}\n\tOutcome: {}".format(wins,losses,pushes,sum(R)))

fig = plt.figure(1,figsize=(12,6))

plt.subplot(221)
plt.plot(CR)
plt.title('Cumulative reward for {} hands'.format(n))

plt.subplot(222)
plt.plot(RTG)
plt.title('Discounted Reward to Go for {} hands'.format(n))

plt.subplot(223)
plt.plot(WALLET)
plt.title('Bankroll over {} hands assuming $5 bet'.format(n))

plt.subplot(224)
plt.plot(RMS)
plt.title('Root Mean Square of Q for {} hands'.format(n))

fig.tight_layout()

Overwriting round.py


In [45]:
states = list(set([q[0:2] for q in iter(Q)]))

In [56]:
s = states[0]
s
qas = {q: Q[q] for q in Q.keys() if q[0:2] == s}
print(qas)
max(qas.keys(), key=lambda k: qas[k])

{(7, 3, 'STAND'): -0.21137440986841682, (7, 3, 'HIT'): 0.1694807532392204}


(7, 3, 'HIT')

In [59]:
import csv

f = open('QL_BasicStrategy_1.csv','w')
writer = csv.writer(f)
for s in states:
    qas = {q: Q[q] for q in Q.keys() if q[0:2] == s}
    oPolicy = max(qas.keys(), key=lambda k: qas[k])
    if oPolicy[0] <= 21:
        writer.writerow(list(oPolicy))
f.close()

So I've got a Q(s,a) policy after 10,000 training episodes, and I've stored it in the same format as the Basic Strategy. Plus, I have a graph that (I hope) shows a measure of the reward slope, smoothing out at any rate, even though it lands on the negative side. The environment isn't REALLY finished yet, I want to add splitting and doubling down to the mix.

So, to do next:

* Compare the learned policy with the optimal one
* Run more training iterations, and compare the results. Don't forget to repeat and average results, at least 10 times
* Decrease the learning rate over time. One option is to keep track of (s,a) and reduce the learning rate by a fraction each time it's used.